# Data Exploration - Outliers

In [1]:
import warnings
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings("ignore")

In [67]:
data = pd.read_csv('data/train.csv')
data

,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
0,2010-03-10 18:00:00,13.1,46.0,0.7578,1387.2,1087.8,1056.0,1742.8,1293.4,2.5,12.0,167.7
1,2010-03-10 19:00:00,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9,2.1,9.9,98.9
2,2010-03-10 20:00:00,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117.0,2.2,9.2,127.1
3,2010-03-10 21:00:00,11.0,62.4,0.7867,1321.0,929.0,1102.9,1536.5,1263.2,2.2,9.7,177.2
4,2010-03-10 22:00:00,11.9,59.0,0.7888,1272.0,852.7,1180.9,1415.5,1132.2,1.5,6.4,121.8
...,...,...,...,...,...,...,...,...,...,...,...,...
7106,2010-12-31 20:00:00,9.2,32.0,0.3871,1000.5,811.2,873.0,909.0,910.5,1.3,5.1,191.1
7107,2010-12-31 21:00:00,9.1,33.2,0.3766,1022.7,790.0,951.6,912.9,903.4,1.4,5.8,221.3
7108,2010-12-31 22:00:00,9.6,34.6,0.4310,1044.4,767.3,861.9,889.2,1159.1,1.6,5.2,227.4
7109,2010-12-31 23:00:00,8.0,40.7,0.4085,952.8,691.9,908.5,917.0,1206.3,1.5,4.6,199.8


In [68]:
cols = [c for c in data.columns if "target" not in c.lower()]
features = data[cols]

Q1 = features.quantile(0.25)
Q3 = features.quantile(0.75)
IQR = Q3 - Q1

In [60]:
((features < (Q1 - 1.5 * IQR)) | (features > (Q3 + 1.5 * IQR))).sum()

absolute_humidity      1
date_time              0
deg_C                 20
relative_humidity      0
sensor_1             129
sensor_2              57
sensor_3             379
sensor_4             352
sensor_5             119
dtype: int64

In [61]:
outliersdf = (features < (Q1 - 1.5 * IQR)) | (features > (Q3 + 1.5 * IQR))
outliersdf = outliersdf[ (outliersdf.sensor_1 == True) | (outliersdf.sensor_2 == True) | (outliersdf.sensor_3 == True) | (outliersdf.sensor_4 == True) | (outliersdf.sensor_5 == True)]

In [62]:
outliersdf.drop(['relative_humidity','absolute_humidity', 'date_time', 'deg_C'], axis=1, inplace=True)

In [63]:
where_outliers_are_df = outliersdf.astype(int).sum(axis=1)
where_outliers_are_df

7       1
8       1
9       1
10      1
11      1
       ..
6955    1
6961    1
6962    1
7090    1
7091    1
Length: 639, dtype: int64

In [64]:
rows_to_delete = where_outliers_are_df.loc[lambda where_outliers_are_df: where_outliers_are_df >= 3]

In [65]:
rows_to_delete

111     3
112     3
121     3
122     3
159     3
169     3
170     3
865     3
998     4
1190    3
1201    3
1502    3
1646    4
1670    3
2174    3
3158    3
4992    3
4993    4
5017    4
5041    4
5375    4
5376    4
5377    3
5424    4
5495    4
5496    4
5497    4
5519    4
5520    4
5521    3
5688    3
5689    3
5690    3
5735    4
5736    4
5759    4
5760    4
6048    3
6168    3
6192    4
6193    4
6194    4
6263    3
6264    3
6266    3
6409    4
6410    3
6672    4
dtype: int64